<a href="https://colab.research.google.com/github/GuiOSousa/studentAnalysis/blob/main/StudentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

O objetivo deste modelo é poupar o tempo de professores e indicar a eles os pontos de maior dificuldade de suas turmas.

O modelo utiliza o Gemini 1.5 Pro para realizar suas análises

#Configuração do Projeto

###Neste tópico, criaremos a base do nosso projeto
Executando as instalações necessárias e acessando os arquivos necessários no nosso Drive.

In [106]:
!pip install -q -U google-generativeai

In [107]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Configuração do Modelo

###Neste tópico, acessaremos os recursos de Google GenerativeAI para criar nosso modelo



Acessando os recursos do Google GenerativeAI utilizando nossa API key (lembre-se de inserir sua chave nas váriaveis do Google Colab e permitir o acesso delas ao código)

In [108]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Escolhendo o modelo.

O modelo escolhido foi o Gemini 1.5 pro, tendo em vista que não precisaremos alterar configurações de segurança e que ele apresentou melhores resultados que a versão 1.0 pro em testes anteriores neste projeto.

In [109]:
model = genai.GenerativeModel(model_name= "models/gemini-1.5-pro-latest")

# Acessando os Dados

###Neste tópico, acessaremos nossa base de dados a partir do nosso Google Drive




Para este passo, divimos nossos dados em duas planilhas: uma com os conteúdos de cada questão e outra com o desempenho dos alunos em cada questão.

Podemos acessá-las através do caminho delas no nosso Google Drive

Note que a linha comentada acessa o arquivo selecionado no input. Para fins de demonstração, utilizei um caminho fixo na váriavel "file_path", embora a linha comentada funcione perfeitamente.

In [110]:
# file_path = input("Digite o caminho do arquivo (a partir de 'Meu Drive'): ")
file_path = "Google AI Studio/NotasFake.xlsx"
full_path = '/content/drive/MyDrive/' + file_path

In [111]:
import pandas as pd

data = pd.read_excel(full_path)

data

,Questao,1,2,3,4,5
0,1,1.00,1.00,0.75,0.50,1.00
1,2,1.00,0.75,0.50,1.00,0.50
2,3,0.50,0.00,1.00,1.00,0.75
3,4,0.75,0.75,0.75,0.50,0.75
4,5,1.00,0.75,1.00,0.75,0.75
5,6,1.00,1.00,1.00,1.00,1.00
6,7,0.75,0.75,1.00,0.75,0.50
7,8,0.50,0.50,0.00,0.00,0.25
8,9,0.50,0.00,0.25,0.00,0.00
9,10,1.00,0.75,1.00,1.00,1.00


In [112]:
#file_path = input("Digite o caminho do arquivo (a partir de 'Meu Drive'): ")
file_path = "Google AI Studio/ConteudosFake.xlsx"
full_path = '/content/drive/MyDrive/' + file_path

In [113]:
content = pd.read_excel(full_path)

content

,Questao,Conteudo
0,1,Operadores condicionais
1,2,For
2,3,While
3,4,For e While
4,5,Operadores condicionais
5,6,Operadores matemáticos
6,7,Operadores condicionais e For
7,8,DoWhile
8,9,While e DoWhile
9,10,For e Operadores matemáticos


No bloco a seguir, uniremos os dados das duas tabelas, substituindo o número da questão por sua respectiva lista de conteúdos.

Note que não utilizo o DataFrame "data" diretamente, mas sim "data.values".

Essa técnica demonstrou melhor interpretação pelo modelo, já que os dados que nos interessam estão em "data.values" (usar ".values" nos retorna somente os valores da tabela, sem as identificações de linhas e colunas)

In [114]:
for index in range(len(content.values)):
  data.iloc[index, 0] = content.iloc[index, 1]

data.values

array([['Operadores condicionais', 1.0, 1.0, 0.75, 0.5, 1.0],
       ['For', 1.0, 0.75, 0.5, 1.0, 0.5],
       ['While', 0.5, 0.0, 1.0, 1.0, 0.75],
       ['For e While', 0.75, 0.75, 0.75, 0.5, 0.75],
       ['Operadores condicionais', 1.0, 0.75, 1.0, 0.75, 0.75],
       ['Operadores matemáticos', 1.0, 1.0, 1.0, 1.0, 1.0],
       ['Operadores condicionais e For', 0.75, 0.75, 1.0, 0.75, 0.5],
       ['DoWhile', 0.5, 0.5, 0.0, 0.0, 0.25],
       ['While e DoWhile', 0.5, 0.0, 0.25, 0.0, 0.0],
       ['For e Operadores matemáticos', 1.0, 0.75, 1.0, 1.0, 1.0]],
      dtype=object)

#Construindo o Prompt

###Nesse tópico, comentarei um pouco sobre a construção do prompt utilizado

Começamos o prompt introduzindo nosso modelo aos nossos dados, armazenados na váriavel "data.values"

Em seguida, explico como os dados estão dispostos na minha tabela (interpretada como uma matriz, ou seja, um vetor composto de outros vetores) e passo algumas diretivas para meu prompt final

Para finalizar, utilizo da técnica "Few Shots" para dar exemplos básicos de como espero que minha resposta seja retornada. A utilização de dois exemplos demonstrou um aumento significativo na qualidade e consistência da resposta do modelo.

In [115]:
prompt = str("Tenho uma tabela que representa uma prova, feita por diferentes alunos. Quero que analise essa tabela:\n\n" +
str(data.values) +
"""\n\nCada vetor corresponde a uma questão. O primeiro valor corresponde ao conteúdo da questão e os valores seguintes ao desempenho de cada aluno na questão
Me diga a nota final de cada aluno, ajustada numa escala de 0 a 10
De acordo com a lista de conteúdos de cada questão, me diga quais conteúdos são uma maior dificuldade para os alunos.
Não inclua códigos na sua resposta

Exemplo de entrada 1:
  [[“Trigonometria e Teorema de Bhaskara”, 0.5, 0.25, 0.5],
  [“Teorema de Bhaskara e Inequações”, 1, 0.75, 1],
  [“Trigonometria e Inequações”], 0.25, 0,5, 0.5]

Exemplo de saída 1:
  A nota final de cada aluno foi:
  Aluno nº 1: (0.5 + 1 + 0.25) /3 = 5,83
  Aluno n° 2: (0.25 + 0.75 + 0.5) /3 = 5
  Aluno nº 3: (0.5 + 1 + 0.5) /3 = 6,67

  O tema de maior dificuldade foi trigonometria.
  ** Insira uma sugestão ao professor de como lidar com essas dificuldades


Exemplo de entrada 2:
  [[“Globalização”, 1, 1, 0.75, 1, 1],
  [“Movimentos de imigração”, 1, 0.75, 1, 1, 1],
  [“Mudanças climáticas e Movimentos de imigração”], 1, 1, 1, 1, 1],
  [“Mudanças climáticas e Topografia”, 0.25, 0.5, 0.5, 0, 0.75],
  [“Intervenções socioeconômicas”, 0.75, 1, 0.75, 1, 1],
  [“Migração de Cérebros”, 0.75, 0, 0, 0.5, 0.25],
  [“Topografia e Movimentos de imigração”, 0.25, 0.5, 0.5, 0, 0.75]]

Exemplo de saída 2:
  A nota final de cada aluno foi:
  Aluno nº 1: (1 + 1 + 1 + 0.25 + 0.75 + 0.75 + 0.25) /7 = 7,14
  Aluno n° 2: (1 + 0.75 + 1 + 0.5 + 1 + 0 + 0.5) /7 = 6,79
  ...
  Aluno nº 5: (1 + 1 + 1 + 0.75 + 1 + 0.25 + 0.75) /7 = 8,21

  Os temas de maior dificuldade foram topografia e migração de cérebros.
  ** Insira uma sugestão ao professor de como lidar com essas dificuldades

""")

Forma final do prompt:

In [116]:
print(prompt)

Tenho uma tabela que representa uma prova, feita por diferentes alunos. Quero que analise essa tabela:

[['Operadores condicionais' 1.0 1.0 0.75 0.5 1.0]
 ['For' 1.0 0.75 0.5 1.0 0.5]
 ['While' 0.5 0.0 1.0 1.0 0.75]
 ['For e While' 0.75 0.75 0.75 0.5 0.75]
 ['Operadores condicionais' 1.0 0.75 1.0 0.75 0.75]
 ['Operadores matemáticos' 1.0 1.0 1.0 1.0 1.0]
 ['Operadores condicionais e For' 0.75 0.75 1.0 0.75 0.5]
 ['DoWhile' 0.5 0.5 0.0 0.0 0.25]
 ['While e DoWhile' 0.5 0.0 0.25 0.0 0.0]
 ['For e Operadores matemáticos' 1.0 0.75 1.0 1.0 1.0]]

Cada vetor corresponde a uma questão. O primeiro valor corresponde ao conteúdo da questão e os valores seguintes ao desempenho de cada aluno na questão
Me diga a nota final de cada aluno, ajustada numa escala de 0 a 10
De acordo com a lista de conteúdos de cada questão, me diga quais conteúdos são uma maior dificuldade para os alunos.
Não inclua códigos na sua resposta

Exemplo de entrada 1:
  [[“Trigonometria e Teorema de Bhaskara”, 0.5, 0.25, 0.5

#Gerando a Resposta

###Nesse tópico, enviaremos o prompt montando anteriormente para nosso modelo

In [117]:
from IPython.display import display
from IPython.display import Markdown

response = model.generate_content(prompt)
display(Markdown(response.text))

## Análise da Prova

**Notas dos Alunos (0-10):**

* Aluno 1: 8.5
* Aluno 2: 6.5
* Aluno 3: 7.75
* Aluno 4: 6.75
* Aluno 5: 7.5

**Temas de Maior Dificuldade:**

* DoWhile
* While e DoWhile

**Sugestão ao Professor:**

Os alunos demonstram dificuldades com laços de repetição, principalmente  "DoWhile" e a combinação "While e DoWhile". Seria interessante **revisar esses conceitos**, com foco em suas particularidades e diferenças em relação a outros laços.

Algumas sugestões:

* **Explicar a lógica de funcionamento do "DoWhile",** que executa o bloco de código pelo menos uma vez antes de verificar a condição, diferenciando-o do "While".
* **Criar exercícios práticos** que reforcem a aplicação de cada tipo de laço, com exemplos claros e objetivos.
* **Promover a resolução de problemas em grupo,** incentivando a colaboração e o aprendizado entre os alunos.
* **Utilizar recursos visuais e analogias** para facilitar a compreensão dos conceitos, tornando o aprendizado mais intuitivo.
* **Oferecer exemplos de código** que demonstrem as diferentes utilizações de "While" e "DoWhile", tanto individualmente quanto em conjunto.

Ao abordar esses pontos, o professor pode auxiliar os alunos a superarem as dificuldades com laços de repetição, consolidando o aprendizado de forma eficaz. 


#Conclusões

O modelo conseguiu compreender bem o problema e nos deu análises bem satisfatórias. Com mais tempo e melhor polimento do código (pequenas verificações no prompt, mudanças no tratamento de dados e etc.) acredito que esta possa ser uma ferramenta muito útil no dia a dia de professores de qualquer área de ensino.